In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from functools import partial
from tqdm.notebook import tqdm
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from transformers_gradients import text_classification, html_heatmap, normalize_sum_to_1


tf.config.list_physical_devices()

Metal device set to: Apple M1 Pro


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

2023-04-30 14:48:58,892:[connectionpool.py:1003->_new_conn()]:DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2023-04-30 14:48:59,045:[connectionpool.py:456->_make_request()]:DEBUG: https://huggingface.co:443 "HEAD /distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json HTTP/1.1" 200 0
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
2023-04-30 14:49:00,346:[connectionpool.py:1003->_new_conn()]:DEBUG: Starting new HTTPS connection (1): huggingface.co:443
2023-04-30 14:49:00,511:[connectionpool.py:456->_make_request()]:DEBUG: https://huggingface.co:443 "HEAD /distilbert-base-uncased-finetuned-sst-2-english/resolve/main/tokenizer_config.jso

In [4]:
x = ["Like four times a year I rediscover Björk and listen to her full discography"]
y = [1]

In [5]:
a_batch = [
    f(model, x, y, tokenizer=tokenizer)[0]
    for f in tqdm(
        [
            text_classification.gradient_norm,
            text_classification.gradient_x_input,
            text_classification.integrated_gradients,
            partial(
                text_classification.smooth_grad,
                config=dict(explain_fn="GradNorm"),
            ),
            partial(
                text_classification.smooth_grad,
                config=dict(explain_fn="GradXInput"),
            ),
            text_classification.smooth_grad,
            partial(
                text_classification.noise_grad,
                config=dict(explain_fn="GradNorm"),
            ),
            partial(
                text_classification.noise_grad,
                config=dict(explain_fn="GradXInput"),
            ),
            text_classification.noise_grad,
            partial(
                text_classification.fusion_grad,
                config=dict(explain_fn="GradNorm"),
            ),
            partial(
                text_classification.fusion_grad,
                config=dict(explain_fn="GradXInput"),
            ),
            # The last one is very slow for 1 sample, but it get bitter as batch size is maximized.
            text_classification.fusion_grad,
        ]
    )
]

  0%|          | 0/12 [00:00<?, ?it/s]

2023-04-30 14:49:00.891263: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 

In [ ]:
a_batch = [(a, normalize_sum_to_1(b)) for a, b in a_batch]

In [ ]:
html_heatmap(
    a_batch,
    labels=[
        "Gradient Norm",
        "Gradient X Input",
        "Integrated Gradients",
        "Smooth Grad + Gradient Norm",
        "Noise Grad + Gradient Norm",
        "FusionGrad + Gradient Norm",
        "FusionGrad + Gradient X Input",
        "FusionGrad + Gradient X Input",
        "FusionGrad + Gradient X Input",
        "FusionGrad + Integrated Gradients",
        "FusionGrad + Integrated Gradients",
        "FusionGrad + Integrated Gradients",
    ],
    config=dict(color_mapping_strategy="global"),
)